In [38]:
from datasets.dataloader import DataLoader
from model import Trainer
from utils.utils import create_folders
from batch_gen import BatchGenerator

import torch 

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## DataLoader : 

In [31]:
class Args(): 
    def __init__(self, *args, **kwargs):
        self.train_data = 'bslcp'
        self.test_data = 'bslcp'
        self.i3d_training = 'i3d_kinetics_bslcp_981'
        self.num_in_frames = 16
        self.features_dim = 1024
        self.weights = 'opt'
        self.regression = 0 
        self.feature_normalization = 0
        self.eval_use_CP = 0

        self.action = 'train'
        self.seed = 0 
        self.refresh = 'store_true'
        ## MSTCN : 
        self.num_stages = 4
        self.num_layers = 10 
        self.num_f_maps = 64
        self.features_dim = 1024
        self.bz = 8 ## ?? 
        self.lr = 0.0005
        self.num_epochs = 50
        self.extract_epoch = 10 
        self.classification_threshold = 0.5

        ## save model : 
        self.use_pseudo_labels = 'store_true'
        self.pretrained = False
        self.folder = ''
        
args = Args()

In [32]:
# load train dataset and test dataset

print(f'Load train data: {args.train_data}')
train_loader = DataLoader(args, args.train_data, 'train')
print(f'Load test data: {args.test_data}')
test_loader = DataLoader(args, args.test_data, 'test')

Load train data: bslcp
Load test data: bslcp


In [33]:
## Some infos : 
print("number of class : ", train_loader.num_classes)
print("cross entropy loss weigths : ", train_loader.weights)
print('number of videos in train : ', len(train_loader.vid_list))

number of class :  2
cross entropy loss weigths :  [0.11247607877029446, 0.8875239212297056]
number of videos in train :  5413


## Trainer : 

In [34]:
model_load_dir, model_save_dir, results_save_dir = create_folders(args)
print(model_save_dir)

Saved options to ./exps//models/classification/traindata_bslcp/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_0\opt.txt
./exps//models/classification/traindata_bslcp/i3d_kinetics_bslcp_981/supervised/4_10_64_1024_8_0.0005_weighted_opt/seed_0


In [35]:
print(f'Start training.')
trainer = Trainer(
    args.num_stages,
    args.num_layers,
    args.num_f_maps,
    args.features_dim,
    train_loader.num_classes,
    device,
    train_loader.weights,
    model_save_dir
    )

Start training.


In [37]:
eval_args = [
    args,
    model_save_dir,
    results_save_dir,
    test_loader.features_dict,
    test_loader.gt_dict,
    test_loader.eval_gt_dict,
    test_loader.vid_list,
    args.num_epochs,
    device,
    'eval',
    args.classification_threshold,
]

In [44]:
batch_gen = BatchGenerator(
        train_loader.num_classes,
        train_loader.gt_dict,
        train_loader.features_dict,
        train_loader.eval_gt_dict
        )

batch_gen.read_data(train_loader.vid_list)

In [45]:
trainer.train(
    model_save_dir,
    batch_gen,
    args.num_epochs,
    args.bz,
    args.lr,
    device,
    eval_args,
    pretrained=model_load_dir)

  1%|          | 5/702 [00:00<00:15, 45.00it/s]


[E1 / train]: epoch loss = 2.4099,   acc = 63.95, mean F1B = 61.46, mean_F1S = 27.77


100%|██████████| 702/702 [00:13<00:00, 51.40it/s]


[E1 / eval]: epoch loss = 2.4513,   acc = 74.01, mean F1B = 65.98, mean_F1S = 46.99



  1%|          | 6/702 [00:00<00:13, 51.28it/s]


[E2 / train]: epoch loss = 2.2747,   acc = 67.37, mean F1B = 67.62, mean_F1S = 33.79


100%|██████████| 702/702 [00:13<00:00, 50.35it/s]


[E2 / eval]: epoch loss = 2.4131,   acc = 69.61, mean F1B = 64.13, mean_F1S = 34.97



KeyboardInterrupt: 